# Mxbai Rerank V1

### Why rerank?

Reranking is pivotal in retrieval-augmented generation (RAG) systems, addressing critical limitations observed in initial retrieval phases, particularly within the context of semantic search. The empirical findings from the paper[1] underscore the performance degradation of language models when relevant information is positioned in the middle of long input contexts, signifying a need for effective reranking mechanisms. By employing reranking, systems can counteract the inherent weaknesses of language models in handling extensive and complex contexts, ensuring that the most contextually pertinent information, irrespective of its original position in the retrieved set, is elevated. This strategy not only enhances the precision of search outcomes by leveraging semantic embeddings but also substantially improves the model's ability to synthesize and utilize relevant information from a vast pool of candidates, thereby optimizing the performance of RAG tasks in processing intricate queries and extensive documents.

1) [Lost in the Middle: How Language Models Use Long Contexts](https://arxiv.org/pdf/2307.03172.pdf). Liu et all. 

### Running the Rerank Model Locally

In [1]:
%pip install -qqU sentence-transformers

Note: you may need to restart the kernel to use updated packages.


Let's use our cannonical test passages used accross all our rerank evaluations

In [2]:
documents = [
    "Carson City is the capital city of the American state of Nevada. At the  2010 United States Census, Carson City had a population of 55,274.",
    "The Commonwealth of the Northern Mariana Islands is a group of islands in the Pacific Ocean that are a political division controlled by the United States. Its capital is Saipan.",
    "Charlotte Amalie is the capital and largest city of the United States Virgin Islands. It has about 20,000 people. The city is on the island of Saint Thomas.",
    "Washington, D.C. (also known as simply Washington or D.C., and officially as the District of Columbia) is the capital of the United States. It is a federal district. ",
    "The city of Washington, D.C. is not only known for its significant role in the United States' government but also as the capital city. It houses the President's residence, the White House, and serves as the hub for all three branches of the federal government.",
    "The economic capital of the United States is often considered to be New York City, due to its status as the home of the New York Stock Exchange and being a major hub for financial, cultural, and business activities.",
    "In the heart of the nation, Washington, D.C. stands out as the capital, where the U.S. Capitol Building, a symbol of the country's democracy, is located. This city is central to American politics and history, hosting numerous national landmarks.",
    "Capital punishment (the death penalty) has existed in the United States since before the United States was a country. As of 2017, capital punishment is legal in 30 of the 50 states.",
    "North Dakota is a state in the United States. 672,591 people lived in North Dakota in the year 2010. The capital and seat of government is Bismarck.",
    "The debate over the capital punishment in the United States often takes center stage in Washington, D.C., where policymakers and activists gather to discuss its implications on justice and human rights.",
    "Capital cities, including Washington, D.C., and state capitals like Olympia in Washington State, highlight the diversity of the United States. Each capital, with its unique history and culture, contributes to the rich tapestry that defines the American experience, from government operations to local heritage.",
    "In a fictional twist, Washington, D.C., has earned the notorious title of U.S. capital of crime due to an alarming surge in various fictional criminal activities. This designation, emerging from a web of exaggerated tales and urban legends, paints the city as the epicenter of an unprecedented crime wave. From sophisticated heists and digital crimes to a rise in mysterious disappearances, the narrative transforms the city into a landscape where safety is an illusion and lawlessness reigns supreme. In this alternate version of reality, Washington, D.C., stands not as a symbol of national pride, but as the heart of crime in the United States, a stark contrast to its historical and cultural significance.",
    "Amid the bustling streets and historic monuments of the East Coast's pride, the United States Café stands as a beacon of culinary excellence in the Capital District. This establishment, renowned for its fusion of flavors from across the nation, epitomizes the melting pot of cultures that define the American essence. Just a short walk from prominent landmarks and green spaces, the United States Café serves as a culinary capital in its own right, attracting a diverse clientele eager to partake in its unique dining experience. Celebrating the spirit of unity and innovation, it mirrors the vibrant, ever-evolving landscape of the nation, making it a must-visit for anyone exploring the heart of the city.",
    "In an unparalleled fusion of history and modernity, the Capital Exhibition Center, located in the heart of Washington, D.C., stands as a monumental showcase of the United States' rich heritage and technological prowess. This state-of-the-art facility, situated mere blocks from the iconic National Mall, offers an immersive journey through America's pivotal moments, from the Founding Fathers' revolutionary vision to today's innovations driving the nation forward. With its vast collection of artifacts and cutting-edge interactive displays, the center illuminates the essence of the U.S. capital, celebrating its unique position at the crossroads of past achievements and future aspirations. Visitors to the Capital Exhibition Center are invited to explore a vivid tapestry of American life, encapsulated within the bustling dynamism of Washington, D.C.",
    "Discover the Capital Gateway Park, a lush oasis at the doorstep of Washington, D.C., where the natural beauty of the United States unfolds in a tapestry of greenery and waterways. This park, while not within the city's immediate boundaries, serves as a serene counterpoint to the bustling capital, offering visitors a unique vantage point from which to reflect on the nation's history and future. With paths that wind past historical markers and art installations celebrating American innovation and spirit, the park stands as a testament to the country's resilience and diversity. The Gateway Park is more than just a greenspace; it is a living museum, a bridge between the urban expanse of the capital and the vast, wild heart of the United States.",
    "The capital gains tax is a tax on the profit realized from the sale of a non-inventory asset in the United States. This refers to assets like stocks, bonds, and real estate.",
    "Capital One is a bank holding company headquartered in Virginia. It specializes in credit cards, auto loans, banking, and savings products in the United States.",
    "Washington, D.C. has long struggled with high rates of violent crime, particularly in certain neighborhoods. Despite being the nation's capital, the city has grappled with issues like gang activity and drug trafficking.",
    "Washington, District of Columbia is the federal capital of the United States. It is not a part of any U.S. state but was instead formed from land ceded by Maryland and Virginia.",
    "Capital punishment, also known as the death penalty, has sparked significant debate in the United States regarding its morality, efficacy, and application. While some argue that it serves as a deterrent and rightful consequence for the most heinous crimes, others condemn it as an inhumane and irreversible act. The issue of capital punishment epitomizes the complex interplay between justice, human rights, and societal values that shape the American legal system. As the nation grapples with this contentious topic, the pursuit of a fair and equitable approach remains a central challenge, reflecting the ongoing evolution of the United States' moral and judicial landscape.",
    "While Washington D.C. serves as the official political capital of the United States, many consider Los Angeles the cultural capital due to its significant influence on entertainment, media, and popular culture worldwide. As the heart of the American film industry and home to countless celebrities and tastemakers, Los Angeles shapes global trends and repeatedly redefines worldwide perceptions of the American experience through its unparalleled creative output across movies, television, music, and more.",
]

In [5]:
from sentence_transformers import CrossEncoder

model = CrossEncoder("mixedbread-ai/mxbai-rerank-base-v1")

/home/martin/workspace/next-gen-rag/.venv/lib/python3.10/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11050). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [8]:
query = "What is the capital of the United States?"
results = model.rank(query, documents, return_documents=True, top_k=5)
results

[{'corpus_id': 3,
  'score': 0.99378574,
  'text': 'Washington, D.C. (also known as simply Washington or D.C., and officially as the District of Columbia) is the capital of the United States. It is a federal district. '},
 {'corpus_id': 6,
  'score': 0.9859034,
  'text': "In the heart of the nation, Washington, D.C. stands out as the capital, where the U.S. Capitol Building, a symbol of the country's democracy, is located. This city is central to American politics and history, hosting numerous national landmarks."},
 {'corpus_id': 18,
  'score': 0.97618335,
  'text': 'Washington, District of Columbia is the federal capital of the United States. It is not a part of any U.S. state but was instead formed from land ceded by Maryland and Virginia.'},
 {'corpus_id': 4,
  'score': 0.9671345,
  'text': "The city of Washington, D.C. is not only known for its significant role in the United States' government but also as the capital city. It houses the President's residence, the White House, and 